In [17]:
import numpy as np
import pandas as pd
from scipy.optimize import Bounds
from copy import copy
from scipy.optimize import OptimizeResult, minimize
from optimparallel import minimize_parallel
from montecarlo import MonteCarlo
from ECIRModel import ECIRModel

In [18]:
class Calibration:
    def __init__(self, model_class, n: int, m: int, r0: float, model_params: dict, optimize_args: tuple, calibrate_exact=False, seed=93756826):
        """
        Initialize the calibration class.
        """
        self.n = n  # Number of simulation paths
        self.m = m  # Time steps per simulation
        self.r0 = r0  # Initial interest rate
        self.seed = seed
        self.model_class = model_class  
        self.model_params = model_params  
        self.optimize_args = optimize_args  
        self.calibrate_exact = calibrate_exact  

    def _calculate_error(self, optimize_params: np.array, Ts: np.array, prices: np.array) -> float:
        model_params = copy(self.model_params)
        for arg, param in zip(self.optimize_args, optimize_params):
            if arg == 'r':
                model_params[arg] = int(param)  
            else:
                model_params[arg] = param

        model = self.model_class(**model_params)
        mc = MonteCarlo(model, self.r0, 1, self.m, self.n)

        errors = []
        for T, market_price in zip(Ts, prices):
            if self.calibrate_exact:
                simulated_price = model.exact_solution(self.r0, T)
            else:
                simulated_price, _ = mc.price_estimates()
            errors.append((simulated_price - market_price)**2)

        return np.sqrt(np.mean(errors))



    def calibrate(self, initial_values: tuple, Ts: np.array, prices: np.array, bounds: Bounds):
        error_function = lambda params: self._calculate_error(params, Ts, prices)
        result = minimize(error_function, initial_values, method='L-BFGS-B', bounds=bounds, options={'maxfun': 500, 'ftol': 1e-6})
        return result


In [19]:
if __name__ == "__main__":
    # Load data and prepare
    df = pd.read_csv("Data Folder/DGS_30.csv", index_col=0)
    df.index = pd.to_datetime(df.index)
    df.index.name = 'DATE'

    selected_date = '2019-04-17'
    yields = df.loc[selected_date].astype(float) / 100
    
    yield_data = pd.DataFrame({
        'Yield': yields.values,
        'Maturity': np.arange(1, len(yields) + 1)
    })
    
    yield_data["Cum. Yield"] = yield_data["Yield"] * yield_data["Maturity"]
    yield_data["Price"] = np.exp(-yield_data["Cum. Yield"])
    
    prices = yield_data["Price"].values.reshape(-1, 1)
    Ts = yield_data["Maturity"].values
    
    initial_model_params = {
        "kappa": 0.5,
        "mu_r": 0.03,
        "sigma": 0.03,
        "mu": 0,
        "gamma": 0.01,
        "r": 10,
        "p": 0.5
    }
    
    bounds = Bounds(
        [0.001, 0.001, 0.001, 0, 0.001, 0.1, 0.001],
        [5, 0.1, 0.5, 1, 0.1, 100, 0.999]
    )

    calibrator = Calibration(
        model_class=ECIRModel, 
        n=100, m=26, r0=0.001,
        model_params=initial_model_params, optimize_args=('kappa', 'mu_r', 'sigma', 'mu', 'gamma', 'r', 'p')
    )
    
    optimal_params = calibrator.calibrate(
        initial_values=(initial_model_params['kappa'], initial_model_params['mu_r'], 
                        initial_model_params['sigma'], initial_model_params['mu'], 
                        initial_model_params['gamma'], initial_model_params['r'], initial_model_params['p']),
        Ts=Ts,
        prices=prices,
        bounds=bounds
    )
    print(optimal_params)

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 0.3619282222664506
        x: [ 5.000e-01  3.000e-02  3.000e-02  0.000e+00  1.000e-02
             1.000e+01  5.000e-01]
      nit: 1
      jac: [-1.250e+05 -2.962e+05 -1.843e+05 -1.653e+05 -6.299e+05
            -3.022e+05 -2.440e+05]
     nfev: 168
     njev: 21
 hess_inv: <7x7 LbfgsInvHessProduct with dtype=float64>


Optimization Results
-------------------
Success: Yes

Objective Function Value: 0.3619

Optimized Parameters:

kappa: 0.5000

mu_r: 0.0300

sigma: 0.0300

mu: 0.0000

gamma: 0.0100

r: 10.0000

p: 0.5000